<a href="https://colab.research.google.com/github/Rakshithbodakuntla/NLP1/blob/main/BPE_on_a_toy_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from collections import defaultdict

toy_corpus = [
    "low", "low", "low", "low", "low",
    "lowest", "lowest",
    "newer", "newer", "newer", "newer", "newer", "newer",
    "wider", "wider", "wider",
    "new", "new"
]
corpus = [" ".join(list(word)) + " _" for word in toy_corpus]
corpus = [tuple(token.split(" ")) for token in corpus]

def get_stats(corpus):
    pairs = defaultdict(int)
    for word in corpus:
        for i in range(len(word)-1):
            pairs[(word[i], word[i+1])] += 1
    return pairs

def merge_pair(pair, corpus):
    new_corpus = []
    replacement = "".join(pair)
    for word in corpus:
        new_word = []
        i = 0
        while i < len(word):
            if i < len(word)-1 and (word[i], word[i+1]) == pair:
                new_word.append(replacement)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_corpus.append(tuple(new_word))
    return new_corpus

vocab = set([sym for word in corpus for sym in word])
print("Initial vocab:", vocab)

for step in range(1, 4):
    pairs = get_stats(corpus)
    best = max(pairs, key=pairs.get)
    corpus = merge_pair(best, corpus)
    vocab.add("".join(best))
    print(f"\nStep {step}: merge {best} → {''.join(best)}")
    print("Corpus sample:", corpus[:8])
    print("Updated vocab:", vocab)


Initial vocab: {'t', 'i', 'o', 'l', 'w', 'e', 'd', '_', 's', 'n', 'r'}

Step 1: merge ('e', 'r') → er
Corpus sample: [('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', 'e', 's', 't', '_'), ('l', 'o', 'w', 'e', 's', 't', '_'), ('n', 'e', 'w', 'er', '_')]
Updated vocab: {'er', 't', 'i', 'o', 'l', 'w', 'e', 'd', '_', 's', 'n', 'r'}

Step 2: merge ('er', '_') → er_
Corpus sample: [('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', 'e', 's', 't', '_'), ('l', 'o', 'w', 'e', 's', 't', '_'), ('n', 'e', 'w', 'er_')]
Updated vocab: {'er', 't', 'i', 'o', 'l', 'w', 'e', 'd', '_', 's', 'er_', 'n', 'r'}

Step 3: merge ('n', 'e') → ne
Corpus sample: [('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', '_'), ('l', 'o', 'w', 'e', 's', 't', '_'), ('l', 'o', 'w', 'e', 's', 't', '_'), ('ne', 'w', 'er_')]
U

In [4]:
from collections import defaultdict

toy_corpus = [
    "low", "low", "low", "low", "low",
    "lowest", "lowest",
    "newer", "newer", "newer", "newer", "newer", "newer",
    "wider", "wider", "wider",
    "new", "new"
]

def get_stats(corpus):
    pairs = defaultdict(int)
    for word in corpus:
        for i in range(len(word)-1):
            pairs[(word[i], word[i+1])] += 1
    return pairs

def merge_pair(pair, corpus):
    new_corpus = []
    replacement = "".join(pair)
    for word in corpus:
        new_word = []
        i = 0
        while i < len(word):
            if i < len(word)-1 and (word[i], word[i+1]) == pair:
                new_word.append(replacement)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_corpus.append(new_word)
    return new_corpus

class BPE:
    def __init__(self, num_merges):
        self.num_merges = num_merges
        self.merges = []

    def fit(self, corpus_words):
        corpus = [list(word) + ["_"] for word in corpus_words]
        for i in range(self.num_merges):
            pairs = get_stats(corpus)
            if not pairs:
                break
            best = max(pairs, key=pairs.get)
            corpus = merge_pair(best, corpus)
            self.merges.append(best)
            print(f"Step {i+1}: merge {best} → {''.join(best)} (vocab size={len(set(x for w in corpus for x in w))})")
        self.corpus = corpus

    def segment(self, word):
        tokens = list(word) + ["_"]
        for pair in self.merges:
            i = 0
            while i < len(tokens)-1:
                if (tokens[i], tokens[i+1]) == pair:
                    tokens[i:i+2] = ["".join(pair)]
                else:
                    i += 1
        return tokens

print("=== Q3.2 Mini-BPE Learner ===")
toy_bpe = BPE(num_merges=10)
toy_bpe.fit(toy_corpus)

test_words = ["new", "newer", "lowest", "widest", "newestest"]
for w in test_words:
    print(f"{w} → {toy_bpe.segment(w)}")


=== Q3.2 Mini-BPE Learner ===
Step 1: merge ('e', 'r') → er (vocab size=11)
Step 2: merge ('er', '_') → er_ (vocab size=11)
Step 3: merge ('n', 'e') → ne (vocab size=11)
Step 4: merge ('ne', 'w') → new (vocab size=11)
Step 5: merge ('l', 'o') → lo (vocab size=10)
Step 6: merge ('lo', 'w') → low (vocab size=10)
Step 7: merge ('new', 'er_') → newer_ (vocab size=11)
Step 8: merge ('low', '_') → low_ (vocab size=12)
Step 9: merge ('w', 'i') → wi (vocab size=11)
Step 10: merge ('wi', 'd') → wid (vocab size=10)
new → ['new', '_']
newer → ['newer_']
lowest → ['low', 'e', 's', 't', '_']
widest → ['wid', 'e', 's', 't', '_']
newestest → ['new', 'e', 's', 't', 'e', 's', 't', '_']


In [6]:
import re
from collections import defaultdict

paragraph = """Data engineering collects and prepares data for analysis.
It involves cleaning, transforming, and moving data between systems.
Good data pipelines make analytics and machine learning reliable and repeatable.
Engineers design systems for scale, latency, and fault tolerance."""

words = re.findall(r"\b\w+\b", paragraph.lower())

def get_stats(corpus):
    pairs = defaultdict(int)
    for word in corpus:
        for i in range(len(word)-1):
            pairs[(word[i], word[i+1])] += 1
    return pairs

def merge_pair(pair, corpus):
    new_corpus = []
    replacement = "".join(pair)
    for word in corpus:
        new_word = []
        i = 0
        while i < len(word):
            if i < len(word)-1 and (word[i], word[i+1]) == pair:
                new_word.append(replacement)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_corpus.append(new_word)
    return new_corpus

class BPE:
    def __init__(self, num_merges):
        self.num_merges = num_merges
        self.merges = []
        self.vocab = set()

    def fit(self, corpus_words):
        corpus = [list(word) + ["_"] for word in corpus_words]
        vocab = set(sym for w in corpus for sym in w)

        for i in range(self.num_merges):
            pairs = get_stats(corpus)
            if not pairs:
                break
            best = max(pairs, key=pairs.get)
            self.merges.append(best)
            corpus = merge_pair(best, corpus)
            vocab.add("".join(best))
            print(f"Step {i+1}: merge {best} → {''.join(best)}")
        self.vocab = vocab
        self.corpus = corpus

    def segment(self, word):
        tokens = list(word) + ["_"]
        for pair in self.merges:
            i = 0
            while i < len(tokens)-1:
                if (tokens[i], tokens[i+1]) == pair:
                    tokens[i:i+2] = ["".join(pair)]
                else:
                    i += 1
        return tokens

para_bpe = BPE(num_merges=30)
para_bpe.fit(words)

print("\nTop 5 merges:", ["".join(p) for p in para_bpe.merges[:5]])
longest = sorted(para_bpe.vocab, key=lambda x: len(x), reverse=True)[:5]
print("5 longest tokens:", longest)

for w in ["data", "transforming", "fault", "pipelines", "reliable"]:
    print(f"{w} → {para_bpe.segment(w)}")


Step 1: merge ('i', 'n') → in
Step 2: merge ('a', 'n') → an
Step 3: merge ('s', '_') → s_
Step 4: merge ('l', 'e') → le
Step 5: merge ('a', 't') → at
Step 6: merge ('d', '_') → d_
Step 7: merge ('at', 'a') → ata
Step 8: merge ('in', 'g') → ing
Step 9: merge ('ing', '_') → ing_
Step 10: merge ('an', 'd_') → and_
Step 11: merge ('d', 'ata') → data
Step 12: merge ('data', '_') → data_
Step 13: merge ('e', 'n') → en
Step 14: merge ('in', 'e') → ine
Step 15: merge ('r', 'e') → re
Step 16: merge ('f', 'o') → fo
Step 17: merge ('fo', 'r') → for
Step 18: merge ('y', 's') → ys
Step 19: merge ('le', '_') → le_
Step 20: merge ('en', 'g') → eng
Step 21: merge ('eng', 'ine') → engine
Step 22: merge ('engine', 'e') → enginee
Step 23: merge ('enginee', 'r') → engineer
Step 24: merge ('o', 'l') → ol
Step 25: merge ('re', 'p') → rep
Step 26: merge ('for', '_') → for_
Step 27: merge ('an', 'a') → ana
Step 28: merge ('ana', 'l') → anal
Step 29: merge ('t', '_') → t_
Step 30: merge ('r', 'an') → ran

Top 